In [43]:
#Third Year Paper Code
import numpy as np
import random 

In [122]:
# Simulation Setting
# Based on DO (1999) lab setting
# with u = 100
# start with production good in storage
# constant gain learning with gamma = 0.7

In [123]:
#Basic Parameters
u = 100
β = 0.9
c_1 = 1
c_2 = 4
c_3 = 9
t = 1
γ = 0.7

In [ ]:
#initial value for Q
#When initial condition is production good in storage
#based on rational expectation
shock = 0
Q_12 = 65 + shock
Q_13 = 103 + shock
Q_11 = Q_12 + u
Q_21 = 256 + shock
Q_23 = 244 + shock
Q_22 = Q_23 + u
Q_31 = 290 + shock
Q_32 = 230 + shock
Q_33 = Q_31 + u

In [96]:
# Create agents
# Agents have there type which same as there consumption type, they also have production type
class Agent:
    agentCount = 0

    def __init__(self, id, consume_type, product_type):
        self.id = id
        self.consume_type = consume_type
        self.product_type = product_type
        self.storage_type = product_type
        self.opposite_storage = 0
        self.trading_result = False
        self.trading = False
        self.original_storage = self.storage_type

        #generate type 1 agents' Q(precieved value for holding good 1 , 2 , 3)
        if self.consume_type == 1:
            self.Q_2 = 65 + random.randint(1, 50)
        if self.consume_type == 1:
            self.Q_3 = 103 + random.randint(1, 50)
        if self.consume_type == 1:
            self.Q_1 = 9999
            
        #generate type 2 agents' Q(precieved value for holding good 1 , 2 , 3)
        if self.consume_type == 2:
            self.Q_1 = 256 + random.randint(1, 50)
        if self.consume_type == 2:
            self.Q_3 = 244 + random.randint(1, 50)
        if self.consume_type == 2:
            self.Q_2 = 9999
            
        #generate type 3 agents' Q(precieved value for holding good 1 , 2 , 3)
        if self.consume_type == 3:
            self.Q_1 = 290 + random.randint(1, 50)
        if self.consume_type == 3:
            self.Q_2 = 230 + random.randint(1, 50)
        if self.consume_type == 3:
            self.Q_3 = 9999

        # other characteristics wait to be added
        Agent.agentCount += 1

    def displayAgent(self):
        print("Agent id:", self.id, "consume:", self.consume_type, ",produce:", self.product_type, ",storage:",
              self.storage_type, "Q_1:", self.Q_1, "Q_2:", self.Q_2, "Q_3:", self.Q_3,"opposite:",self.opposite_storage,"trading:",self.trading,"trading_reuslt:",self.trading_result)

        


In [97]:
# matching process
# update agent's information of what situation he is facing
def match(mylist):
    random.shuffle(mylist)
    matchlist = dict()
    index = list(range(0, len(mylist), 2))
    for i in index:
        matchlist[mylist[i]] = mylist[i + 1]
    for key in matchlist:
        update_opposite_storage(key,matchlist[key]) 
        key.original_storage = key.storage_type #easy to update Q at the end of the round
        matchlist[key].original_storage = matchlist[key].storage_type
    return matchlist

# update what good the other agent is holding
def update_opposite_storage(agent1, agent2):
    storage_1 = agent1.storage_type
    storage_2 = agent2.storage_type
    agent1.opposite_storage = storage_2
    agent2.opposite_storage = storage_1

In [100]:
#get the parameters for decision making and updating Q
def get_Q_hold(agent):
    if agent.storage_type == 1:
        return agent.Q_1
    elif agent.storage_type == 2:
        return agent.Q_2
    elif agent.storage_type == 3:
        return agent.Q_3

def get_original_Q_hold(agent):
    if agent.original_storage == 1:
        return agent.Q_1
    elif agent.original_storage == 2:
        return agent.Q_2
    elif agent.original_storage == 3:
        return agent.Q_3

def set_Q_hold(agent,update_value):
    if agent.storage_type == 1:
        agent.Q_1 = update_value
    elif agent.storage_type == 2:
        agent.Q_2 = update_value
    elif agent.storage_type == 3:
        agent.Q_3 = update_value
        
def set_original_Q_hold(agent,update_value):
    if agent.original_storage == 1:
        agent.Q_1 = update_value
    elif agent.original_storage == 2:
        agent.Q_2 = update_value
    elif agent.original_storage == 3:
        agent.Q_3 = update_value

def get_Q_trade(agent):
    if agent.opposite_storage == 1:
        return agent.Q_1
    elif agent.opposite_storage == 2:
        return agent.Q_2
    elif agent.opposite_storage == 3:
        return agent.Q_3

def get_C_hold(agent):
    if agent.storage_type == 1:
        return c_1
    elif agent.storage_type == 2:
        return c_2
    elif agent.storage_type == 3:
        return c_3

def get_C_trade(agent):
    if agent.opposite_storage == 1:
        return c_1
    elif agent.opposite_storage == 2:
        return c_2
    elif agent.opposite_storage == 3:
        return c_3


In [101]:
# trading decision by each agent
def transaction_decision(agent):
    Q_hold = get_Q_hold(agent)
    Q_trade = get_Q_trade(agent)
    C_hold = get_C_hold(agent)
    C_trade = get_C_trade(agent)

    if β * Q_hold - C_hold < β * Q_trade - C_trade:
        agent.trading = True
    else:
        agent.trading = False
        
# trading results 
def produce_outcome(matched_list):
    trade_outcome = list()
    for key in matched_list.keys():
        if key.trading == True and matched_list[key].trading == True:
            trade_outcome.append(True)
            key.trading_result = True
            matched_list[key].trading_result = True
        else:
            trade_outcome.append(False)
            key.trading_result = False
            matched_list[key].trading_result = False
    return trade_outcome



In [121]:
# based on trading result, updates Q and storage

def update_trading(agent):
    if(agent.trading_result == False): #if there is trade
        v_prime = get_C_hold(agent)*(-1) + β * get_Q_hold(agent)
        updated_value = get_Q_hold(agent) + γ * (v_prime - get_Q_hold(agent))
        set_Q_hold(agent,updated_value)
    else: #if trade happened 
        if agent.consume_type == agent.opposite_storage: #agent gets his own consumption good
            agent.storage_type = agent.product_type #agent eats the consumption good and produces another production good 
            v_prime = get_C_hold(agent) * (-1) + β * get_Q_hold(agent) + u
            updated_value = get_original_Q_hold(agent) + γ * (v_prime - get_original_Q_hold(agent))
            set_original_Q_hold(agent,updated_value)
        else: #agent does not get his cons good
            agent.storage_type = agent.opposite_storage 
            v_prime = get_C_hold(agent) *(-1) + β * get_Q_hold(agent)
            updated_value = get_original_Q_hold(agent) + γ * (v_prime - get_original_Q_hold(agent))
            set_original_Q_hold(agent, updated_value)

In [117]:
# Global behavior recorder

def global_a1_s2_f3(mylist,count_down,count_up): #agent 1 holding 2 facing opportunity for 3
    result = list()
    for ele in mylist:
        if (ele.consume_type == 1 and ele.storage_type == 2 and ele.opposite_storage == 3):
            count_down += 1
            if (ele.trading == True):
                count_up += 1

    result.append(count_up)
    result.append(count_down)
    return result

def global_a2_s3_f1(mylist,count_down,count_up): #agent 2 holding 3 facing opportunity for 1
    result = list()
    for ele in mylist:
        if (ele.consume_type == 2 and ele.storage_type == 3 and ele.opposite_storage == 1):
            count_down += 1
            if (ele.trading == True):
                count_up += 1

    result.append(count_up)
    result.append(count_down)
    return result

def global_a3_s1_f2(mylist,count_down,count_up): #agent 3 holding 1 facing opportunity for 2
    result = list()
    for ele in mylist:
        if (ele.consume_type == 3 and ele.storage_type == 1 and ele.opposite_storage == 2):
            count_down += 1
            if (ele.trading == True):
                count_up += 1

    result.append(count_up)
    result.append(count_down)
    return result

In [120]:
# main code

if __name__ == '__main__':
    u = 100
    β = 0.9
    c_1 = 1
    c_2 = 4
    c_3 = 9
    t = 1

    mylist = list()
    agent1 = Agent(1, 1, 2)
    mylist.append(agent1)
    agent2 = Agent(2, 1, 2)
    mylist.append(agent2)
    agent3 = Agent(3, 1, 2)
    mylist.append(agent3)
    agent4 = Agent(4, 1, 2)
    mylist.append(agent4)
    agent5 = Agent(5, 1, 2)
    mylist.append(agent5)
    agent6 = Agent(6, 1, 2)
    mylist.append(agent6)
    agent7 = Agent(7, 1, 2)
    mylist.append(agent7)
    agent8 = Agent(8, 1, 2)
    mylist.append(agent8)
    agent9 = Agent(9, 2, 3)
    mylist.append(agent9)
    agent10 = Agent(10, 2, 3)
    mylist.append(agent10)
    agent11 = Agent(11, 2, 3)
    mylist.append(agent11)
    agent12 = Agent(12, 2, 3)
    mylist.append(agent12)
    agent13 = Agent(13, 2, 3)
    mylist.append(agent13)
    agent14 = Agent(14, 2, 3)
    mylist.append(agent14)
    agent15 = Agent(15, 2, 3)
    mylist.append(agent15)
    agent16 = Agent(16, 2, 3)
    mylist.append(agent16)
    agent17 = Agent(17, 3, 1)
    mylist.append(agent17)
    agent18 = Agent(18, 3, 1)
    mylist.append(agent18)
    agent19 = Agent(19, 3, 1)
    mylist.append(agent19)
    agent20 = Agent(20, 3, 1)
    mylist.append(agent20)
    agent21 = Agent(21, 3, 1)
    mylist.append(agent21)
    agent22 = Agent(22, 3, 1)
    mylist.append(agent22)
    agent23 = Agent(23, 3, 1)
    mylist.append(agent23)
    agent24 = Agent(24, 3, 1)
    mylist.append(agent24)

    game_continue = True

    count_round = 0
    sum_up123 = 0
    sum_down123 = 0
    sum_up231 = 0
    sum_down231 = 0
    sum_up312 = 0
    sum_down312 = 0
    
    while(count_round < 130):
        count_round += 1
        matched_list = match(mylist)

        for key in matched_list.keys():
            transaction_decision(key)
            transaction_decision(matched_list[key])

        global_r1_2_3 = global_a1_s2_f3(mylist,0,0)
        sum_up123 = sum_up123 + global_r1_2_3[0]
        sum_down123 = sum_down123 + global_r1_2_3[1]
        
        global_r2_3_1 = global_a2_s3_f1(mylist,0,0)
        sum_up231 = sum_up231 + global_r2_3_1[0]
        sum_down231 = sum_down231 + global_r2_3_1[1]
        
        global_r3_1_2 = global_a3_s1_f2(mylist,0,0)
        sum_up312 = sum_up312 + global_r3_1_2[0]
        sum_down312 = sum_down312 + global_r3_1_2[1]

        for ele in mylist:
            ele.displayAgent()

        produce_outcome(matched_list)

        for key in matched_list.keys():
            update_trading(key)
            update_trading(matched_list[key])

        print(count_round)

        stopping_draw = random.uniform(0, 1)
        if stopping_draw > 0.01:
            game_continue = True
        else:
            game_continue = False

    print("Agent 1 with good 2 facing good 3:", sum_up123/sum_down123)
    print("Agent 2 with good 3 facing good 1:", sum_up231/sum_down231)
    print("Agent 3 with good 1 facing good 2:", sum_up312/sum_down312)
    

Agent id: 1 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 105 Q_3: 153 opposite: 2 trading: False trading_reuslt: False
Agent id: 3 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 77 Q_3: 130 opposite: 2 trading: False trading_reuslt: False
Agent id: 4 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 71 Q_3: 142 opposite: 2 trading: False trading_reuslt: False
Agent id: 6 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 72 Q_3: 138 opposite: 2 trading: False trading_reuslt: False
Agent id: 23 consume: 3 ,produce: 1 ,storage: 1 Q_1: 297 Q_2: 246 Q_3: 9999 opposite: 3 trading: True trading_reuslt: False
Agent id: 15 consume: 2 ,produce: 3 ,storage: 3 Q_1: 304 Q_2: 9999 Q_3: 262 opposite: 1 trading: True trading_reuslt: False
Agent id: 12 consume: 2 ,produce: 3 ,storage: 3 Q_1: 260 Q_2: 9999 Q_3: 279 opposite: 2 trading: True trading_reuslt: False
Agent id: 7 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 68 Q_3: 113 opposite: 3 trading: True trading_reuslt: False
Agent id: 20 

Agent id: 23 consume: 3 ,produce: 1 ,storage: 1 Q_1: 275.95695707 Q_2: 246 Q_3: 9999 opposite: 1 trading: False trading_reuslt: False
Agent id: 22 consume: 3 ,produce: 1 ,storage: 1 Q_1: 426.76604928 Q_2: 277 Q_3: 9999 opposite: 1 trading: False trading_reuslt: True
Agent id: 10 consume: 2 ,produce: 3 ,storage: 1 Q_1: 289 Q_2: 9999 Q_3: 259.8860702 opposite: 3 trading: False trading_reuslt: True
Agent id: 3 consume: 1 ,produce: 2 ,storage: 3 Q_1: 9999 Q_2: 96.243 Q_3: 100.278 opposite: 1 trading: True trading_reuslt: False
Agent id: 1 consume: 1 ,produce: 2 ,storage: 3 Q_1: 9999 Q_2: 132.6095295 Q_3: 153 opposite: 1 trading: True trading_reuslt: True
Agent id: 14 consume: 2 ,produce: 3 ,storage: 1 Q_1: 264 Q_2: 9999 Q_3: 239.0686211 opposite: 3 trading: False trading_reuslt: True
Agent id: 11 consume: 2 ,produce: 3 ,storage: 3 Q_1: 310.67455 Q_2: 9999 Q_3: 267.05 opposite: 2 trading: True trading_reuslt: True
Agent id: 7 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 138.36332099999

Agent id: 2 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 143.20520418 Q_3: 160.23066 opposite: 3 trading: True trading_reuslt: True
Agent id: 13 consume: 2 ,produce: 3 ,storage: 3 Q_1: 230.1243 Q_2: 9999 Q_3: 192.00756420000002 opposite: 2 trading: True trading_reuslt: False
Agent id: 1 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 190.526862435 Q_3: 191.541003585 opposite: 3 trading: False trading_reuslt: True
Agent id: 17 consume: 3 ,produce: 1 ,storage: 2 Q_1: 238.69845755606934 Q_2: 263 Q_3: 9999 opposite: 2 trading: False trading_reuslt: True
Agent id: 6 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 115.1963952 Q_3: 171.932888976 opposite: 2 trading: False trading_reuslt: True
Agent id: 12 consume: 2 ,produce: 3 ,storage: 3 Q_1: 260 Q_2: 9999 Q_3: 342.86310994662335 opposite: 2 trading: True trading_reuslt: False
Agent id: 5 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 133.4349737258 Q_3: 179.4894 opposite: 3 trading: True trading_reuslt: False
Agent id: 4 consum

Agent id: 19 consume: 3 ,produce: 1 ,storage: 1 Q_1: 206.4993132518394 Q_2: 277.4548251060848 Q_3: 9999 opposite: 1 trading: False trading_reuslt: False
Agent id: 22 consume: 3 ,produce: 1 ,storage: 1 Q_1: 451.2519770479622 Q_2: 277 Q_3: 9999 opposite: 1 trading: False trading_reuslt: True
Agent id: 8 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 158.65849531592144 Q_3: 90.48851 opposite: 3 trading: False trading_reuslt: False
Agent id: 13 consume: 2 ,produce: 3 ,storage: 3 Q_1: 230.1243 Q_2: 9999 Q_3: 136.83475831721944 opposite: 2 trading: True trading_reuslt: False
Agent id: 5 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 146.80881411773998 Q_3: 207.8770954941762 opposite: 1 trading: True trading_reuslt: True
Agent id: 17 consume: 3 ,produce: 1 ,storage: 1 Q_1: 291.28956552714453 Q_2: 292.21702826032373 Q_3: 9999 opposite: 2 trading: False trading_reuslt: True
Agent id: 11 consume: 2 ,produce: 3 ,storage: 3 Q_1: 325.7498328835 Q_2: 9999 Q_3: 332.13698234500004 opposite: 2 tr

Agent id: 21 consume: 3 ,produce: 1 ,storage: 1 Q_1: 269.68929905520696 Q_2: 302.88196495878844 Q_3: 9999 opposite: 1 trading: False trading_reuslt: False
Agent id: 1 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 348.360247716743 Q_3: 191.541003585 opposite: 1 trading: True trading_reuslt: True
Agent id: 15 consume: 2 ,produce: 3 ,storage: 1 Q_1: 260.9512689449955 Q_2: 9999 Q_3: 288.1530212558258 opposite: 2 trading: True trading_reuslt: False
Agent id: 5 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 164.78222930018046 Q_3: 233.37593310736656 opposite: 2 trading: False trading_reuslt: True
Agent id: 6 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 182.77418490107152 Q_3: 171.932888976 opposite: 2 trading: False trading_reuslt: True
Agent id: 2 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 167.95933480652425 Q_3: 196.70668668401999 opposite: 1 trading: True trading_reuslt: False
Agent id: 9 consume: 2 ,produce: 3 ,storage: 1 Q_1: 301.9641825233112 Q_2: 9999 Q_3: 284.52187

...

Agent id: 8 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 170.8130024055631 Q_3: 56.867010614013765 opposite: 1 trading: True trading_reuslt: False
Agent id: 16 consume: 2 ,produce: 3 ,storage: 3 Q_1: 199.14507997833164 Q_2: 9999 Q_3: 228.4644353347883 opposite: 1 trading: False trading_reuslt: True
Agent id: 10 consume: 2 ,produce: 3 ,storage: 1 Q_1: 288.32237097838777 Q_2: 9999 Q_3: 268.6263208206147 opposite: 3 trading: False trading_reuslt: True
Agent id: 9 consume: 2 ,produce: 3 ,storage: 3 Q_1: 231.24476665797926 Q_2: 9999 Q_3: 238.48805260105593 opposite: 2 trading: True trading_reuslt: False
Agent id: 2 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 77.9037651716545 Q_3: 235.18592547136564 opposite: 3 trading: True trading_reuslt: False
Agent id: 24 consume: 3 ,produce: 1 ,storage: 1 Q_1: 259.7334551378745 Q_2: 330.33087450474295 Q_3: 9999 opposite: 2 trading: True trading_reuslt: False
Agent id: 20 consume: 3 ,produce: 1 ,storage: 2 Q_1: 259.6422716744216 Q_2: 197.52549

Agent id: 21 consume: 3 ,produce: 1 ,storage: 1 Q_1: 301.88965985439233 Q_2: 242.76942584753272 Q_3: 9999 opposite: 3 trading: True trading_reuslt: True
Agent id: 6 consume: 1 ,produce: 2 ,storage: 3 Q_1: 9999 Q_2: 274.8693379567411 Q_3: 216.76324187875707 opposite: 1 trading: True trading_reuslt: False
Agent id: 7 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 262.07977502566246 Q_3: 97.97157720176915 opposite: 3 trading: False trading_reuslt: True
Agent id: 15 consume: 2 ,produce: 3 ,storage: 3 Q_1: 290.53440726023285 Q_2: 9999 Q_3: 332.51276667369297 opposite: 2 trading: True trading_reuslt: True
Agent id: 3 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 159.88666717103956 Q_3: 227.71011047170768 opposite: 3 trading: True trading_reuslt: False
Agent id: 14 consume: 2 ,produce: 3 ,storage: 3 Q_1: 255.2294339244373 Q_2: 9999 Q_3: 271.12704059250296 opposite: 2 trading: True trading_reuslt: False
Agent id: 9 consume: 2 ,produce: 3 ,storage: 3 Q_1: 231.24476665797926 Q_2: 9999 Q_3

Agent id: 4 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 150.7115870115047 Q_3: 207.95316227547102 opposite: 3 trading: True trading_reuslt: True
Agent id: 23 consume: 3 ,produce: 1 ,storage: 1 Q_1: 197.66582027973027 Q_2: 290.40318145364347 Q_3: 9999 opposite: 2 trading: True trading_reuslt: False
Agent id: 6 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 232.3304903987854 Q_3: 305.396655476374 opposite: 1 trading: True trading_reuslt: False
Agent id: 5 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 267.3564044188286 Q_3: 316.47500323454597 opposite: 1 trading: True trading_reuslt: False
Agent id: 24 consume: 3 ,produce: 1 ,storage: 1 Q_1: 332.70249335579587 Q_2: 318.84197362349 Q_3: 9999 opposite: 2 trading: False trading_reuslt: True
Agent id: 15 consume: 2 ,produce: 3 ,storage: 3 Q_1: 290.53440726023285 Q_2: 9999 Q_3: 249.8511014633517 opposite: 1 trading: True trading_reuslt: False
Agent id: 21 consume: 3 ,produce: 1 ,storage: 1 Q_1: 301.41363113149947 Q_2: 242.76942584

Agent id: 23 consume: 3 ,produce: 1 ,storage: 2 Q_1: 258.2492858576679 Q_2: 300.33835037738663 Q_3: 9999 opposite: 1 trading: False trading_reuslt: True
Agent id: 18 consume: 3 ,produce: 1 ,storage: 1 Q_1: 203.68793389385831 Q_2: 204.45766819265918 Q_3: 9999 opposite: 2 trading: False trading_reuslt: False
Agent id: 20 consume: 3 ,produce: 1 ,storage: 1 Q_1: 200.979432696817 Q_2: 257.05194845819005 Q_3: 9999 opposite: 2 trading: True trading_reuslt: True
Agent id: 1 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 227.0993823377933 Q_3: 233.9840297680499 opposite: 1 trading: True trading_reuslt: False
Agent id: 15 consume: 2 ,produce: 3 ,storage: 1 Q_1: 312.9542865962298 Q_2: 9999 Q_3: 266.3556705122384 opposite: 3 trading: False trading_reuslt: True
Agent id: 14 consume: 2 ,produce: 3 ,storage: 3 Q_1: 255.2294339244373 Q_2: 9999 Q_3: 291.27795341687437 opposite: 1 trading: False trading_reuslt: True
Agent id: 17 consume: 3 ,produce: 1 ,storage: 1 Q_1: 191.79828754653343 Q_2: 152.1425

Agent id: 14 consume: 2 ,produce: 3 ,storage: 1 Q_1: 255.2294339244373 Q_2: 9999 Q_3: 232.02473394547192 opposite: 1 trading: False trading_reuslt: True
Agent id: 10 consume: 2 ,produce: 3 ,storage: 1 Q_1: 237.34028771474954 Q_2: 9999 Q_3: 216.44485579581178 opposite: 1 trading: False trading_reuslt: True
127
Agent id: 4 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 242.6770464616678 Q_3: 154.5620070679253 opposite: 1 trading: True trading_reuslt: False
Agent id: 24 consume: 3 ,produce: 1 ,storage: 1 Q_1: 422.7023942501877 Q_2: 318.84197362349 Q_3: 9999 opposite: 2 trading: False trading_reuslt: True
Agent id: 23 consume: 3 ,produce: 1 ,storage: 1 Q_1: 239.47183584763116 Q_2: 308.30464186275134 Q_3: 9999 opposite: 2 trading: True trading_reuslt: False
Agent id: 7 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 253.37555160024334 Q_3: 97.97157720176915 opposite: 1 trading: True trading_reuslt: True
Agent id: 5 consume: 1 ,produce: 2 ,storage: 2 Q_1: 9999 Q_2: 234.57927722868376 Q_

Agent id: 1 consume: 1 ,produce: 2 ,storage: 2 Q1: 72


In [ ]:
#whats next:



#3.1 when type 1 agent faced trading opportunity for good 3
#3.2 when type 2 agent faced trading opportunity for good 1
#3.3 when type 3 agent faced trading opportunity for good 2 


#5
